# Stock Market Trend Analysis - Phase 1
## Notebook 2: Data Quality Assessment

**Author:** Enerita  
**Date:** December 2025  
**Project:** Stock Market Trend Analysis - Global Investment Partners  
**Purpose:** Comprehensive data quality analysis and issue resolution

## Setup: Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully")

## Part 1: Load Datasets

In [ ]:
data_path = "../data/raw/"

company_info = pd.read_csv(data_path + 'company_info.csv')
stock_prices = pd.read_csv(data_path + 'stock_prices.csv')
stock_indicators = pd.read_csv(data_path + 'stock_prices_with_indicators.csv')
market_indices = pd.read_csv(data_path + 'market_indices.csv')

print("All datasets loaded successfully")

## Part 2: Missing Values Analysis

In [ ]:
print("Missing Values Summary\n")

datasets = {
    'Company Info': company_info,
    'Stock Prices': stock_prices,
    'Stock Indicators': stock_indicators,
    'Market Indices': market_indices
}

for name, df in datasets.items():
    total_cells = df.shape[0] * df.shape[1]
    missing_cells = df.isnull().sum().sum()
    missing_pct = (missing_cells / total_cells) * 100
    
    print(f"{name}:")
    print(f"  Total cells: {total_cells:,}")
    print(f"  Missing cells: {missing_cells:,}")
    print(f"  Missing percentage: {missing_pct:.2f}%\n")

In [ ]:
print("Detailed Missing Values by Column (Stock Indicators)\n")

missing_indicators = stock_indicators.isnull().sum()
missing_pct_indicators = (missing_indicators / len(stock_indicators)) * 100

missing_df = pd.DataFrame({
    'Column': missing_indicators.index,
    'Missing_Count': missing_indicators.values,
    'Missing_Percentage': missing_pct_indicators.values
})

missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
    
    os.makedirs('../visuals/eda', exist_ok=True)
    
    plt.figure(figsize=(12, 6))
    top_missing = missing_df.head(15)
    plt.barh(top_missing['Column'], top_missing['Missing_Percentage'])
    plt.xlabel('Missing Percentage (%)')
    plt.title('Top 15 Columns with Missing Values')
    plt.tight_layout()
    plt.savefig('../visuals/eda/missing_values.png', dpi=300, bbox_inches='tight')
    print("\nVisualization saved: visuals/eda/missing_values.png")
else:
    print("No missing values found in primary dataset")

## Part 3: Data Type Verification

In [ ]:
print("Stock Indicators Data Types:\n")
print(stock_indicators.dtypes)

In [ ]:
print("Converting date columns to datetime\n")

for name, df in datasets.items():
    if 'date' in df.columns:
        original_type = df['date'].dtype
        df['date'] = pd.to_datetime(df['date'])
        print(f"{name}: {original_type} -> datetime64")

stock_prices['date'] = pd.to_datetime(stock_prices['date'])
stock_indicators['date'] = pd.to_datetime(stock_indicators['date'])
market_indices['date'] = pd.to_datetime(market_indices['date'])

print("\nAll date columns converted to datetime format")

In [ ]:
print("Numeric Columns Verification\n")

numeric_cols = stock_indicators.select_dtypes(include=[np.number]).columns
print(f"Total numeric columns: {len(numeric_cols)}")
print(f"Examples: {list(numeric_cols[:10])}")

## Part 4: Duplicate Records Check

In [ ]:
print("Duplicate Records Check\n")

for name, df in datasets.items():
    duplicates = df.duplicated().sum()
    print(f"{name}:")
    print(f"  Total records: {len(df):,}")
    print(f"  Duplicate records: {duplicates:,}")
    print(f"  Duplicate percentage: {(duplicates/len(df)*100):.2f}%\n")

In [ ]:
print("Ticker-Date Combination Duplicates\n")

ticker_date_duplicates = stock_indicators.duplicated(subset=['ticker', 'date']).sum()
print(f"Stock Indicators: {ticker_date_duplicates} ticker-date duplicates")

if ticker_date_duplicates > 0:
    print("\nExample duplicates:")
    print(stock_indicators[stock_indicators.duplicated(subset=['ticker', 'date'], keep=False)].head())

## Part 5: Date Range and Continuity Check

In [ ]:
print("Date Range Coverage\n")

date_datasets = {
    'Stock Prices': stock_prices,
    'Stock Indicators': stock_indicators,
    'Market Indices': market_indices
}

for name, df in date_datasets.items():
    min_date = df['date'].min()
    max_date = df['date'].max()
    date_range_days = (max_date - min_date).days
    unique_dates = df['date'].nunique()
    
    print(f"{name}:")
    print(f"  Start date: {min_date.date()}")
    print(f"  End date: {max_date.date()}")
    print(f"  Date range: {date_range_days} days")
    print(f"  Unique trading days: {unique_dates}")
    print(f"  Coverage: {(unique_dates/date_range_days*100):.1f}% of calendar days\n")

In [ ]:
print("Records per Stock\n")

records_per_ticker = stock_indicators.groupby('ticker')['date'].count().sort_values(ascending=False)
print(f"Mean records per stock: {records_per_ticker.mean():.0f}")
print(f"Min records: {records_per_ticker.min()}")
print(f"Max records: {records_per_ticker.max()}")
print(f"Std deviation: {records_per_ticker.std():.0f}")

print("\nStocks with varying record counts:")
print(records_per_ticker.head(10))

In [ ]:
print("Date Gap Analysis (per ticker)\n")

sample_tickers = stock_indicators['ticker'].unique()[:3]
for ticker in sample_tickers:
    ticker_data = stock_indicators[stock_indicators['ticker'] == ticker].sort_values('date')
    date_diffs = ticker_data['date'].diff().dt.days
    gaps = date_diffs[date_diffs > 7]
    
    print(f"{ticker}:")
    print(f"  Total records: {len(ticker_data)}")
    print(f"  Date gaps > 7 days: {len(gaps)}")
    if len(gaps) > 0:
        print(f"  Max gap: {gaps.max()} days\n")
    else:
        print()

## Part 6: Outlier Detection

In [ ]:
print("Price Data Outliers\n")

negative_prices = (stock_indicators[['open', 'high', 'low', 'close', 'volume']] < 0).sum()
print("Negative values found:")
if negative_prices.sum() > 0:
    print(negative_prices[negative_prices > 0])
else:
    print("None")

In [ ]:
stock_indicators['daily_return'] = stock_indicators.groupby('ticker')['close'].pct_change()
extreme_returns = stock_indicators[abs(stock_indicators['daily_return']) > 0.5]

print(f"\nExtreme daily returns (>50% change): {len(extreme_returns)}")
if len(extreme_returns) > 0:
    print("\nExample extreme returns:")
    print(extreme_returns[['ticker', 'date', 'close', 'daily_return']].head())

In [ ]:
print("Technical Indicator Outliers\n")

if 'rsi_14' in stock_indicators.columns:
    rsi_outliers = stock_indicators[(stock_indicators['rsi_14'] < 0) | (stock_indicators['rsi_14'] > 100)]
    print(f"RSI outside 0-100 range: {len(rsi_outliers)}")

Q1 = stock_indicators['volume'].quantile(0.25)
Q3 = stock_indicators['volume'].quantile(0.75)
IQR = Q3 - Q1
volume_outliers = stock_indicators[(stock_indicators['volume'] < Q1 - 1.5*IQR) | 
                                   (stock_indicators['volume'] > Q3 + 1.5*IQR)]
print(f"Volume outliers (IQR method): {len(volume_outliers)} ({len(volume_outliers)/len(stock_indicators)*100:.2f}%)")

## Part 7: Target Variable Analysis

In [ ]:
if 'trend_label' in stock_indicators.columns:
    print("Trend Label Distribution\n")
    
    trend_counts = stock_indicators['trend_label'].value_counts()
    trend_pcts = stock_indicators['trend_label'].value_counts(normalize=True) * 100
    
    trend_summary = pd.DataFrame({
        'Count': trend_counts,
        'Percentage': trend_pcts
    })
    print(trend_summary)
    
    max_class_pct = trend_pcts.max()
    min_class_pct = trend_pcts.min()
    imbalance_ratio = max_class_pct / min_class_pct
    
    print(f"\nClass Imbalance Ratio: {imbalance_ratio:.2f}")
    if imbalance_ratio > 2:
        print("WARNING: Significant class imbalance detected")
        print("Recommendation: Consider using class weights or resampling techniques")
    
    plt.figure(figsize=(10, 6))
    trend_counts.plot(kind='bar', color=['green', 'red', 'gray'])
    plt.title('Target Variable Distribution (Trend Labels)')
    plt.xlabel('Trend Label')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('../visuals/eda/target_distribution.png', dpi=300, bbox_inches='tight')
    print("\nVisualization saved: visuals/eda/target_distribution.png")
    
    print("\nMissing Target Values")
    missing_target = stock_indicators['trend_label'].isnull().sum()
    print(f"Missing trend labels: {missing_target} ({missing_target/len(stock_indicators)*100:.2f}%)")
    
else:
    print("WARNING: Target variable 'trend_label' not found in dataset")

## Part 8: Data Quality Summary Report

In [ ]:
print("Data Quality Summary Report\n")

quality_issues = []

total_missing = stock_indicators.isnull().sum().sum()
quality_issues.append({
    'Check': 'Missing Values',
    'Status': 'PASS' if total_missing == 0 else 'ATTENTION',
    'Details': f'{total_missing:,} missing cells',
    'Action': 'Handle with forward fill' if total_missing > 0 else 'None required'
})

duplicates = stock_indicators.duplicated().sum()
quality_issues.append({
    'Check': 'Duplicate Records',
    'Status': 'PASS' if duplicates == 0 else 'FAIL',
    'Details': f'{duplicates} duplicates',
    'Action': 'Remove duplicates' if duplicates > 0 else 'None required'
})

min_records = records_per_ticker.min()
max_records = records_per_ticker.max()
quality_issues.append({
    'Check': 'Date Coverage',
    'Status': 'ATTENTION' if max_records - min_records > 50 else 'PASS',
    'Details': f'Records vary from {min_records} to {max_records}',
    'Action': 'Document variations' if max_records - min_records > 50 else 'None required'
})

if 'trend_label' in stock_indicators.columns:
    missing_target = stock_indicators['trend_label'].isnull().sum()
    quality_issues.append({
        'Check': 'Target Variable',
        'Status': 'PASS' if missing_target == 0 else 'ATTENTION',
        'Details': f'{missing_target} missing labels',
        'Action': 'Remove rows with missing target' if missing_target > 0 else 'None required'
    })

date_correct = pd.api.types.is_datetime64_any_dtype(stock_indicators['date'])
quality_issues.append({
    'Check': 'Data Types',
    'Status': 'PASS' if date_correct else 'FAIL',
    'Details': 'Date column correctly formatted' if date_correct else 'Date needs conversion',
    'Action': 'None required' if date_correct else 'Convert to datetime'
})

quality_df = pd.DataFrame(quality_issues)
print(quality_df.to_string(index=False))

## Part 9: Recommendations and Next Steps

In [ ]:
print("Recommendations and Next Steps\n")

print("Data Cleaning Actions Required:")

actions = []
if total_missing > 0:
    actions.append("Handle missing values using forward fill method")
if duplicates > 0:
    actions.append("Remove duplicate records")
if 'trend_label' in stock_indicators.columns and missing_target > 0:
    actions.append("Remove rows with missing target variable")
actions.append("Verify extreme values and outliers")
actions.append("Create clean modeling dataset")

for i, action in enumerate(actions, 1):
    print(f"{i}. {action}")

print("\nData Quality Score")

passed_checks = sum(1 for issue in quality_issues if issue['Status'] == 'PASS')
total_checks = len(quality_issues)
quality_score = (passed_checks / total_checks) * 100

print(f"Checks passed: {passed_checks}/{total_checks}")
print(f"Data Quality Score: {quality_score:.0f}%")

if quality_score >= 80:
    print("Data quality is GOOD - proceed to EDA")
elif quality_score >= 60:
    print("Data quality is ACCEPTABLE - address issues before modeling")
else:
    print("Data quality is POOR - significant cleanup required")

print("\nReady for Next Step:")
print("Proceed to Notebook 03: Exploratory Data Analysis")
print("After addressing the data quality issues identified above")

In [ ]:
quality_report = {
    'assessment_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_records': len(stock_indicators),
    'total_features': stock_indicators.shape[1],
    'missing_values': total_missing,
    'duplicates': duplicates,
    'quality_score': quality_score,
    'recommendations': actions
}

print("\nQuality assessment complete")
print("Data quality report ready for documentation")